<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone project part 1 : binary classifier







In [2]:
# importing needed library

import numpy as np # linear algebra
import pandas as pd
pd.set_option("display.max_rows", 101)
import os
# print(os.listdir("./input"))
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 15
import seaborn as sns
from collections import Counter
from PIL import Image
import math
import seaborn as sns
from collections import defaultdict
from pathlib import Path
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, AvgPool2D
from keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten ,Dropout, Activation, Conv2D, MaxPool2D , MaxPooling2D, AveragePooling2D, AvgPool2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from keras.regularizers import l2
import tensorflow as tf 

import pickle
from numpy import loadtxt
from keras.models import load_model
import keras
from keras.constraints import max_norm
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model

Using TensorFlow backend.


In [20]:
# importing the data 

train_df = pd.read_csv("./train.csv")

In [4]:
train_df.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [5]:
train = train_df.ImageId_ClassId.str.split('_', expand=True)

In [6]:
train.head()

,0,1
0,0002cc93b.jpg,1
1,0002cc93b.jpg,2
2,0002cc93b.jpg,3
3,0002cc93b.jpg,4
4,00031f466.jpg,1


In [7]:
train['EncodedPixels'] = train_df.EncodedPixels

In [8]:
train.head()

,0,1,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg,2,NaN
2,0002cc93b.jpg,3,NaN
3,0002cc93b.jpg,4,NaN
4,00031f466.jpg,1,NaN


In [9]:
train.columns = ['ImageId','ClassId','EncodedPixels']

In [10]:
train['isdefect'] = train.EncodedPixels.apply(lambda x:0 if x is np.nan else 1)

In [11]:
train.head()

,ImageId,ClassId,EncodedPixels,isdefect
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,1
1,0002cc93b.jpg,2,NaN,0
2,0002cc93b.jpg,3,NaN,0
3,0002cc93b.jpg,4,NaN,0
4,00031f466.jpg,1,NaN,0


In [12]:
train = train.groupby(by='ImageId').sum()

In [13]:
train.head()

,isdefect
ImageId,
0002cc93b.jpg,1
00031f466.jpg,0
000418bfc.jpg,0
000789191.jpg,0
0007a71bf.jpg,1


In [14]:
train.reset_index(inplace=True)

In [15]:
train.isdefect.value_counts()

1    6239
0    5902
2     425
3       2
Name: isdefect, dtype: int64

In [16]:
train.isdefect = train.isdefect.apply(lambda x:1 if x>0 else 0)

In [17]:
# count the isdefect column unique values:
# 1= contain defect 
# 0 = defect-free 

train.isdefect.value_counts()

1    6666
0    5902
Name: isdefect, dtype: int64

In [19]:
# model baseline 
train.isdefect.value_counts(normalize=True)

1    0.530395
0    0.469605
Name: isdefect, dtype: float64

# spliting the trainning data  80% - 20% ( manually implemented due to lack of computing power)

In [23]:
x1_train= train[train.isdefect==0]
len(x1_train)

5902

In [24]:
x1_train=x1_train.sample(n = 4722)
len(x1_train)

4722

In [25]:
x2_train= train[train.isdefect==1]
len(x2_train)

6666

In [26]:
x2_train=x2_train.sample(n = 5333)
x2_train.reset_index(inplace=True,drop= True)
len(x2_train)

5333

In [27]:
new_list=pd.concat([x1_train,x2_train ], ignore_index=True)

In [28]:
len(new_list)

10055

In [29]:
new_list = shuffle(new_list)
new_list.reset_index(inplace=True,drop= True)

In [30]:
new_list.head()

,ImageId,isdefect
0,08808b1a2.jpg,1
1,dcb3d23c8.jpg,1
2,b1639af61.jpg,1
3,c51011137.jpg,1
4,0ff538004.jpg,1


# Preparing the Testing data 

In [31]:
x_list = [x for x in new_list.ImageId]

In [32]:
test=[i for i in train.ImageId if i not in x_list]

In [33]:
len(test)

2513

In [34]:
test_data = []
for image_name in tqdm(test):
    test_data.append(train.set_index(keys='ImageId').loc[image_name])


100%|██████████| 2513/2513 [00:17<00:00, 206.66it/s]


In [35]:
test_df=pd.DataFrame(test_data)

In [36]:
test_df.reset_index(inplace=True)

In [37]:
test_df.head()

,index,isdefect
0,001d3d093.jpg,0
1,0030401a5.jpg,1
2,0046839bd.jpg,1
3,005f19695.jpg,1
4,0095cd374.jpg,1


In [38]:
# reading the images 
test_name = []
for idx, image_name in tqdm(enumerate(test_df["index"])):
    image = cv2.imread('./train_images/' + image_name,cv2.IMREAD_GRAYSCALE )
    test_name.append(image)

2513it [00:31, 79.38it/s] 


In [39]:
# reading the images 
images_list = []
for idx, image_name in tqdm(enumerate(new_list.ImageId)):
    image = cv2.imread('./train_images/' + image_name,cv2.IMREAD_GRAYSCALE )
    images_list.append(image)

10055it [01:45, 155.17it/s]


In [40]:
X_test=test_name
y_test=test_df.isdefect

In [41]:
# change the type of the X and y (test)
y_test = np.array(y_test)
X_test=np.array(X_test).reshape(-1,256,1600,1)

In [42]:
X= images_list
y= new_list.isdefect

In [43]:
# change the type of the X and y (train)
y = np.array(y)
X =np.array(X).reshape(-1,256,1600,1)

# CNN multi classifier model

In [35]:
# instantiate the model
model = tf.keras.Sequential()

# creating convolution layers
model.add(Conv2D(32,kernel_size=(5,5),input_shape=X.shape[1:],kernel_regularizer=l2(0.0005),activation="relu",strides=(2, 2)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(5,5),kernel_regularizer=l2(0.0005),activation="relu",strides=(2, 2)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(5,5),kernel_regularizer=l2(0.0005),activation="relu",strides=(2, 2)))
model.add(MaxPooling2D(pool_size=(2,2)))

# creating convolution layers 
model.add(Flatten())
model.add(Dense(1000))
model.add(Dropout(0.25))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 798, 32)      832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 399, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 198, 64)       51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 99, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 48, 128)        204928    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 24, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)             

In [37]:
#compiling
model.compile(optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"])

In [38]:
#fitting the model 
model.fit(
    x=X,
    y=y,
    batch_size=None,
    epochs=30,
    verbose=1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=6),
             ModelCheckpoint(filepath='final first model.h5', monitor='val_loss', save_best_only=True)]
                ,
    validation_split=0.2,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False)

Train on 8044 samples, validate on 2011 samples
Epoch 1/30
8044/8044 [==============================] - 472s 59ms/sample - loss: 1.1047 - accuracy: 0.7182 - val_loss: 0.4961 - val_accuracy: 0.7548
Epoch 2/30
8044/8044 [==============================] - 491s 61ms/sample - loss: 0.4640 - accuracy: 0.8012 - val_loss: 0.4339 - val_accuracy: 0.8245
Epoch 3/30
8044/8044 [==============================] - 478s 59ms/sample - loss: 0.4459 - accuracy: 0.8207 - val_loss: 0.4693 - val_accuracy: 0.7911
Epoch 4/30
8044/8044 [==============================] - 473s 59ms/sample - loss: 0.3979 - accuracy: 0.8366 - val_loss: 0.4570 - val_accuracy: 0.8498
Epoch 5/30
8044/8044 [==============================] - 463s 58ms/sample - loss: 0.3695 - accuracy: 0.8584 - val_loss: 0.3834 - val_accuracy: 0.8429
Epoch 6/30
8044/8044 [==============================] - 476s 59ms/sample - loss: 0.3127 - accuracy: 0.8780 - val_loss: 0.4725 - val_accuracy: 0.8120
Epoch 7/30
8044/8044 [==============================] - 47

In [22]:
print ("the testing results")
print ("accuracy: 87.62%")
print ("loss: 0.31%")

the testing results
accuracy: 87.62%
loss: 0.31%


# import the model ( to save time )

### place the "final first model.h5" file in same directory of this sheet to save time 

In [44]:
#model= tf.keras.models.load_model('final first model.h5')

In [23]:
#model.summary()

In [24]:
# score = model.evaluate(X_test,y_test, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
# print("%s: %.2f%%" % (model.metrics_names[0], score[0]))